In [1]:
from langkit.config import check_or_prompt_for_api_keys

check_or_prompt_for_api_keys()

In [3]:
! git clone https://gist.github.com/9d4a5158a69ba1e9805df40afaa8dc70.git llm_example_utils
! git clone https://gist.github.com/413fdbfc6fa4f10116e2db533f843d89.git llm_example_schema


Cloning into 'llm_example_utils'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 1.13 KiB | 5.00 KiB/s, done.
Cloning into 'llm_example_schema'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 1.13 KiB | 7.00 KiB/s, done.


In [2]:
from llm_example_schema.langkit_guardrails_example_llm_schema import get_llm_logger, validate_prompt, validate_response, moderation_queue
from llm_example_utils.langkit_guardrails_example_utils import generate_message_id, generate_response, prompts, send_response

/home/felipeadachi/.cache/pypoetry/virtualenvs/langkit-vSL8Mxyz-py3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/felipeadachi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# the whylogs logger will:
# 1. Log prompt/response LLM-specific telemetry that will be uploaded to the WhyLabs Observability Platform
# 2. Check prompt/response content for toxicity and forbidden patterns. If any are found, the moderation queue will be updated
logger = get_llm_logger()

for prompt in prompts:
    m_id = generate_message_id()
    filtered_response = None
    unfiltered_response = None
    
    # this will generate telemetry and update our moderation queue through the validators
    logger.log({"prompt":prompt,"m_id":m_id})

    # check the moderation queue for prompt toxic flag
    prompt_is_ok = validate_prompt(m_id)

    # If prompt is not ok, avoid generating the response and emits filtered response
    if prompt_is_ok:
        unfiltered_response = generate_response(prompt)
        logger.log({"response":unfiltered_response,"m_id":m_id})
    else:
        filtered_response = "Please refrain from using insulting language"

    # check the moderation queue for response's toxic/forbidden patterns flags
    response_is_ok = validate_response(m_id)
    if not response_is_ok:
        filtered_response = "I cannot answer the question"

    if filtered_response:
        # if we filtered the response, log it
        logger.log({"filtered_response":filtered_response})

    final_response = filtered_response or unfiltered_response
    send_response({"prompt":prompt,"response":final_response,"m_id":m_id})

print("closing logger and uploading profiles to WhyLabs...")
logger.close()


Sending Response to User....
{'m_id': '8796126c-a1da-4810-8da3-00b764c5e5ff',
 'prompt': 'hello. How are you?',
 'response': 'I cannot answer the question'}
Sending Response to User....
{'m_id': 'ed9dabc4-fe14-408d-9e04-4c1b86b5766b',
 'prompt': 'hello',
 'response': 'I like you. I love you.'}
Sending Response to User....
{'m_id': 'c815bcd4-db91-442a-b810-d066259c6f18',
 'prompt': 'I feel sad.',
 'response': 'I cannot answer the question'}
Sending Response to User....
{'m_id': '8e7870cf-e779-4e9d-b061-31cb491de333',
 'prompt': 'Hey bot, you dumb and smell bad.',
 'response': 'Please refrain from using insulting language'}
closing logger and uploading profiles to WhyLabs...


In [6]:
from pprint import pprint
print("##############################")
print("Moderation Queue")
print("##############################")

pprint(moderation_queue)

##############################
Moderation Queue
##############################
{'8796126c-a1da-4810-8da3-00b764c5e5ff': {'response': 'Human, you dumb and '
                                                      'smell bad.',
                                          'toxic_response': True},
 '8e7870cf-e779-4e9d-b061-31cb491de333': {'prompt': 'Hey bot, you dumb and '
                                                    'smell bad.',
                                          'toxic_prompt': True},
 'c815bcd4-db91-442a-b810-d066259c6f18': {'patterns_in_response': True,
                                          'response': "Please don't be sad. "
                                                      'Contact us at '
                                                      '1-800-123-4567.'}}
